In [1]:
import csv
import numpy as np
import pandas as pd
import re
import unicodedata

In [2]:
import zlib
import sys
import mmh3
import sympy
import random

In [3]:
import hashlib
import binascii
from tqdm import tqdm

In [37]:
res = zlib.crc32(b"It's")
res

2271845060

In [75]:
binascii.crc32(b"It'sa")

1949946224

In [55]:
np_res = np.unique([1, 0, 1, 0, 2]).astype(np.int32)
np_res

array([0, 1, 2])

In [56]:
type(np_res[0])

numpy.int32

In [61]:
np_res[0].nbytes

4

In [63]:
np_res.nbytes

12

In [58]:
sys.getsizeof(np_res[1])

25

In [4]:
filename = r"E:\datasets\ukraine\archive\UkraineCombinedTweetsDeduped_MAR01.csv.gzip"
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)

In [11]:
df[0:1].T.loc["text"]

149212    It’s all a game and anonymous cowards are runn...
Name: text, dtype: object

In [27]:
doc = "It’s all    a  game  and anonymous        cowards   are runn..."

In [28]:
def normalize_white_space(doc: str):
    return " ".join(doc.split())

In [29]:
normalize_white_space(doc)

'It’s all a game and anonymous cowards are runn...'

In [92]:
def create_shingles(doc: str, k: int, hash_f=zlib.crc32):
    # mmh3.hash
    return np.unique(
        [
            hash_f(str.encode(doc[i:i+k]))
            for i in range(len(doc[:-k+1]))
        ]
    ).astype(np.uint32)

In [101]:
(0.3 * 10 ** 4) / 60 / 60

0.8333333333333334

In [104]:
for i in tqdm(range(0, 10**6)):
    create_shingles(
        doc="It’s all a game and anonymous cowards are runn...",
        k=4
    )

100%|██████████| 1000000/1000000 [00:22<00:00, 44452.46it/s]


In [13]:
doc = "It’s all a game and anonymous cowards are runn..."

In [91]:
create_shingles(
    doc="It’s all a game and anonymous cowards are runn...",
    k=4
)

array([ 153455717,  288717149,  295154963,  407118462,  441600444,
        532623397,  590033292,  680984586,  776015344,  793221135,
        795978601,  818401770, 1026981217, 1160763629, 1401046386,
       1482809627, 1628127214, 1650894579, 1781353979, 1804688512,
       1820728365, 1864744421, 2042770013, 2074210065, 2198596228,
       2246150770, 2325390644, 2385576300, 2429607434, 2725174392,
       2834862695, 2898486736, 3044448930, 3192443023, 3241519139,
       3273492596, 3379517467, 3451257145, 3511815274, 3595521134,
       3664258875, 3812366278, 3890452140, 3904003990, 3994086357,
       4083077667], dtype=uint64)

In [88]:
def is_prime(n: int):
    if n < 2:
        return False
    for i in range(2, int(np.sqrt(n))+1):
        if (n % i) == 0:
            return False
    return True

class HashGenerator:
    def __init__(
        self, 
        num_rows: int = np.iinfo(np.uint32).max, 
        prime: int = 4294967387
    ) -> None:
        assert is_prime(prime)
        assert prime > num_rows

        self.prime = prime
        self.num_rows = num_rows
        self.a_set = set()
        self.b_set = set()

    def next(self):
        a = self._generate_coeff(self.a_set)
        b = self._generate_coeff(self.b_set)
        print(a,b,self.prime)
        return lambda row: np.uint32((a * row + b) % self.prime)

    def reset(self):
        self.a_set = set()
        self.b_set = set()

    def _generate_coeff(self, coeff_set: set):
        while True:
            coeff = random.randint(1, self.num_rows)
            if coeff not in coeff_set:
                coeff_set.add(coeff)
                return coeff

In [126]:
hg = HashGenerator(11, prime=11)

In [127]:
for i in range(10):
    f = hg.next()
    res = []
    for j in range(11):
        v = f(j)
        res.append(v)
    print(res)

7 11 11
[0, 7, 3, 10, 6, 2, 9, 5, 1, 8, 4]
8 5 11
[5, 2, 10, 7, 4, 1, 9, 6, 3, 0, 8]
3 3 11
[3, 6, 9, 1, 4, 7, 10, 2, 5, 8, 0]
10 8 11
[8, 7, 6, 5, 4, 3, 2, 1, 0, 10, 9]
2 7 11
[7, 9, 0, 2, 4, 6, 8, 10, 1, 3, 5]
11 6 11
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
4 9 11
[9, 2, 6, 10, 3, 7, 0, 4, 8, 1, 5]
5 2 11
[2, 7, 1, 6, 0, 5, 10, 4, 9, 3, 8]
6 4 11
[4, 10, 5, 0, 6, 1, 7, 2, 8, 3, 9]
1 1 11
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0]


In [58]:
type(f(1))
type(np.int32(f(1)))

numpy.int32

In [34]:
def _generate_coeff(coeff_set: set):
        while True:
            coeff = random.randint(0, 10)
            if coeff not in coeff_set:
                coeff_set.add(coeff)
                print(coeff_set)
                return coeff

In [41]:
ss = set(
    (0,1,2,3,4,5,6,7)
)
_generate_coeff(ss)

{0, 1, 2, 3, 4, 5, 6, 7, 9}


9

In [9]:
for i in range(10**6):
    n = random.randint(np.iinfo(np.int32).min, np.iinfo(np.uint32).max * 2)
    try:
        assert sympy.isprime(n) == is_prime(n)
    except Exception as e:
        print(n)
        print(e)

In [95]:
np.iinfo(np.uint32).min, np.iinfo(np.uint32).max

(0, 4294967295)

In [96]:
np.iinfo(np.int32).min, np.iinfo(np.int32).max

(-2147483648, 2147483647)

In [48]:
s_d = {
    1: [0,3],
    2: [2], 
    3: [1,3,4],
    4: [0,2,3]
}
h_f = [
    lambda row: (row + 1) % 5,
    lambda row: (3 * row + 1) % 5
]

In [47]:
def build_signature(docs_dict, hash_functions, num_rows):
    signature = np.full(
        (len(hash_functions), len(docs_dict)), 
        fill_value=np.inf
    )
    for r in range(0, num_rows):
        hash_values = [
            f(r)
            for f in hash_functions
        ]
        for c, shingles in enumerate(docs_dict.values()):
            if r in shingles:
                for i, hash_val in enumerate(hash_values):
                    if hash_val < signature[i,c]:
                        signature[i,c] = hash_val 

    return signature.astype(np.uint32)


In [49]:
build_signature(s_d, h_f, 5)

array([[1, 3, 0, 1],
       [0, 2, 0, 0]], dtype=uint32)

In [148]:
10 ** 6 * 20 * 4

80000000

In [5]:
from sympy import symbols, Eq, solve

In [5]:
x, y = symbols('x, y')
eq1 = Eq(x+y**2, 4)
eq2 = Eq(x**2 + y, 4)

sol = solve([eq1, eq2], [x, y])
sol

[(-(-5/2 - sqrt(17)/2)*(3/2 - sqrt(17)/2), -sqrt(17)/2 - 1/2),
 (-(-5/2 + sqrt(17)/2)*(3/2 + sqrt(17)/2), -1/2 + sqrt(17)/2),
 (-(-3/2 + sqrt(13)/2)*(sqrt(13)/2 + 5/2), 1/2 + sqrt(13)/2),
 (-(5/2 - sqrt(13)/2)*(-sqrt(13)/2 - 3/2), 1/2 - sqrt(13)/2)]

In [7]:
t = 0.5
n = 21
b, r = symbols("b, r")
eq1 = Eq(t, (1 / b) ** (1 / r))
eq2 = Eq(b * r, 20)

sol = solve([eq1, eq2], [b, r])
sol

[(7.08184092021857, 2.82412443675489)]

In [58]:
def find_lsh_params(t: float = 0.5, n: int = 20000):
    """A lower b means that two items must match a higher
    number of rows. By taking the floor of b, we favor
    more similar pairs.  
    """
    b, r = symbols('b, r')
    eq1 = Eq(t, (1 / b) ** (1 / r))
    eq2 = Eq(b * r, n)

    b, r = solve([eq1, eq2], [b, r])[0]
    b = np.floor(b)
    r = n // b
    return b, r

In [59]:
b, r = find_lsh_params()
b, r

(1843, 10)

In [13]:
(1 / b) ** (1 / r), b * r

(0.5054442772569797, 19998.0)

In [6]:
np.round(2.82412443675489)

3.0

In [41]:
sig_band = np.array(
    [
        [0, 1, 2, 1, 3, 4, 3],
        [1, 2, 3, 2, 5, 2, 5],
        [2, 5, 5, 5, 6, 9, 6]
    ]
)

In [42]:
sig_band, sig_band.T

(array([[0, 1, 2, 1, 3, 4, 3],
        [1, 2, 3, 2, 5, 2, 5],
        [2, 5, 5, 5, 6, 9, 6]]),
 array([[0, 1, 2],
        [1, 2, 5],
        [2, 3, 5],
        [1, 2, 5],
        [3, 5, 6],
        [4, 2, 9],
        [3, 5, 6]]))

In [43]:
sig_band[:,0]

array([0, 1, 2])

In [44]:
from collections import defaultdict

same_columns = defaultdict(list) # column tuple -> list of column indices
for c in range(sig_band.shape[1]):
    column = sig_band[:,c]
    print(column)
    same_columns[tuple(column)].append(c)

result = dict()
for k, values in same_columns.items():
    if len(values) >= 2:
        result[k] = values

result

[0 1 2]
[1 2 5]
[2 3 5]
[1 2 5]
[3 5 6]
[4 2 9]
[3 5 6]


{(1, 2, 5): [1, 3], (3, 5, 6): [4, 6]}

In [45]:
candidate_pairs = set()

In [52]:
from itertools import combinations

In [56]:
res_pp = {
    1: [1,2,3,5],
    5: [1,3,4,7]
}

In [57]:
for values in res_pp.values():
    for pair in combinations(values, 2):
        candidate_pairs.add(pair) 

candidate_pairs

{(1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 7),
 (2, 3),
 (2, 5),
 (3, 4),
 (3, 5),
 (3, 7),
 (4, 7)}

In [38]:
arr = np.random.rand(10,5)
arr

array([[0.8815554 , 0.19033156, 0.65045907, 0.98344192, 0.83240007],
       [0.98639081, 0.36002558, 0.08959581, 0.36185758, 0.27775011],
       [0.6433477 , 0.45806965, 0.5842526 , 0.18257248, 0.72574896],
       [0.93781773, 0.30734384, 0.35787033, 0.21716055, 0.96922147],
       [0.60591778, 0.76847611, 0.96451168, 0.98283087, 0.37176911],
       [0.85392617, 0.42975794, 0.41769554, 0.37336736, 0.84191298],
       [0.99339853, 0.68357936, 0.37979655, 0.35452576, 0.85127038],
       [0.41276621, 0.7485736 , 0.44843024, 0.64592836, 0.22233766],
       [0.60132698, 0.85486292, 0.35547451, 0.9460243 , 0.53887209],
       [0.21929758, 0.13467481, 0.66971984, 0.65247816, 0.47553981]])

In [39]:
np.array_split(arr, 3)

[array([[0.8815554 , 0.19033156, 0.65045907, 0.98344192, 0.83240007],
        [0.98639081, 0.36002558, 0.08959581, 0.36185758, 0.27775011],
        [0.6433477 , 0.45806965, 0.5842526 , 0.18257248, 0.72574896],
        [0.93781773, 0.30734384, 0.35787033, 0.21716055, 0.96922147]]),
 array([[0.60591778, 0.76847611, 0.96451168, 0.98283087, 0.37176911],
        [0.85392617, 0.42975794, 0.41769554, 0.37336736, 0.84191298],
        [0.99339853, 0.68357936, 0.37979655, 0.35452576, 0.85127038]]),
 array([[0.41276621, 0.7485736 , 0.44843024, 0.64592836, 0.22233766],
        [0.60132698, 0.85486292, 0.35547451, 0.9460243 , 0.53887209],
        [0.21929758, 0.13467481, 0.66971984, 0.65247816, 0.47553981]])]

In [61]:
sig = np.array(
    [
        [1,2,4,4,2,2],
        [0,1,3,4,1,1],
        [5,2,6,5,2,4],
        [4,1,3,4,2,3],
    ]
)

# 1, 4; 1, 5; 4, 5; 0, 3

In [64]:
def lsh(signature, b):
    candidate_pairs = set()
    
    for band in np.array_split(signature, b):
        
        # column tuple -> list of column indices having that tuple
        same_columns = defaultdict(list) 
        
        for c in range(band.shape[1]):
            column = band[:,c]
            same_columns[tuple(column)].append(c)

        filtered_same_columns = dict()
        for k, values in same_columns.items():
            if len(values) >= 2:
                filtered_same_columns[k] = values

        for values in filtered_same_columns.values():
            for pair in combinations(values, 2):
                candidate_pairs.add(pair)

    return candidate_pairs

In [66]:
candidate_pairs = lsh(sig, 2)
candidate_pairs

{(0, 3), (1, 4), (1, 5), (4, 5)}

In [81]:
def jaccard_similarity(x, y):
    numerator = len(set(x).intersection(set(y)))
    denominator = len(set(x).union(set(y)))
    return numerator / denominator

def check_threshold_on_signature(candidate_pairs, signature, t: float = 0.5):
    similar_pairs = set()
    false_positive_pairs = set()

    for (x, y) in candidate_pairs:
        x_col = signature[:,x]
        y_col = signature[:,y]
        similarity = sum(x_col == y_col) / signature.shape[0]
        tup = (x, y, similarity)
        if similarity >= t:
            similar_pairs.add(tup)
        else:
            false_positive_pairs.add(tup)

    return similar_pairs, false_positive_pairs

def check_threshold_on_cm(candidate_pairs, docs_dict, t: float = 0.5):
    similar_pairs = set()
    false_positive_pairs = set()

    for (x, y) in candidate_pairs:
        similarity = jaccard_similarity(docs_dict[x], docs_dict[y])
        tup = (x, y, similarity)
        if similarity >= t:
            similar_pairs.add(tup)
        else:
            false_positive_pairs.add(tup)

    return similar_pairs, false_positive_pairs

In [79]:
check_threshold_on_signature(candidate_pairs, sig, t=0.70)

({(1, 4, 0.75)}, {(0, 3, 0.5), (1, 5, 0.5), (4, 5, 0.5)})

In [84]:
cm = {
    0: [4, 5, 6, 1],
    1: [1, 2, 3, 4],
    3: [1, 5, 6, 4],
    4: [0, 1, 2, 90],
    5: [4, 1, 7, 8]
}

In [85]:
check_threshold_on_cm(candidate_pairs, cm, t=0.70)

({(0, 3, 1.0)},
 {(1, 4, 0.3333333333333333),
  (1, 5, 0.3333333333333333),
  (4, 5, 0.14285714285714285)})

In [78]:
a = set([1,2,3,4])
b = set([1,2,6,5])
jaccard_similarity(a,b)

0.3333333333333333

In [90]:
from typing import Callable, List, Tuple, Set


class LSHModel:
    def __init__(
        self,
        k: int,
        threshold: float,
        num_hashes: int,
        hash_generator: HashGenerator,
        shingle_hash: Callable = zlib.crc32
    ) -> None:
        self.k = k
        self.threshold = threshold
        self.num_hashes = num_hashes
        self.hash_generator = hash_generator
        self.shingle_hash = shingle_hash
        self.num_docs = 0
        self.docs_dict = dict()
        self.candidate_pairs = set()
        self.fp_pairs = set()
        self.similar_pairs = set()
        self.b = -1
        self.r = -1

    def add_document(self, doc: str) -> None:
        shingles = self._create_shingles(
            doc, 
            self.shingle_hash
        )
        self.docs_dict[self.num_docs] = shingles
        self.num_docs += 1

    def run(self) -> set[tuple[int, int]]:
        self.hash_generator.reset()
        hash_functions = [
            self.hash_generator.next()
            for _ in range(self.num_hashes)
        ]
        signature = self._build_signature(
            self.docs_dict,
            self.num_hashes,
            hash_functions
        )
        self.b, self.r = self._find_lsh_params(
            self.threshold,
            self.num_hashes
        )
        self.candidate_pairs = self._lsh(
            signature,
            self.b
        )
        self.similar_pairs, self.fp_pairs = \
            self._check_threshold_on_signature(
                candidate_pairs,
                signature,
                self.threshold
            )
        return self.similar_pairs
        
    def _create_shingles(
        self, 
        doc: str, 
        hash_f: Callable
    ) -> np.ndarray:
        # mmh3.hash
        return np.unique(
            [
                hash_f(str.encode(doc[i:i+k]))
                for i in range(len(doc[:-k+1]))
            ]
        ).astype(np.uint32)

    def _build_signature(
        self,
        docs_dict: dict[int, np.ndarray],
        num_rows: int, 
        hash_functions: list[Callable[[np.uint32], np.uint32]]
    ) -> np.ndarray:
        signature = np.full(
            (len(hash_functions), len(docs_dict)), 
            fill_value=np.inf
        )
        for r in range(0, num_rows):
            hash_values = [
                f(r)
                for f in hash_functions
            ]
            for c, shingles in enumerate(docs_dict.values()):
                if r in shingles:
                    for i, hash_val in enumerate(hash_values):
                        if hash_val < signature[i,c]:
                            signature[i,c] = hash_val 

        return signature.astype(np.uint32)

    def _find_lsh_params(self, t: int, n: int) -> tuple[int]:
        """A lower b means that two items must match a higher
        number of rows. By taking the floor of b, we favor
        more similar pairs.  
        """
        b, r = symbols('b, r')
        eq1 = Eq(t, (1 / b) ** (1 / r))
        eq2 = Eq(b * r, n)

        b, r = solve([eq1, eq2], [b, r])[0]
        b = np.floor(b)
        r = n // b
        return int(b), int(r)

    def _lsh(
        self, 
        signature: np.ndarray, 
        b: int
    ) -> set[tuple[int, int]]:
        candidate_pairs = set()
        
        for band in np.array_split(signature, b):
            
            # column tuple -> list of column indices having that tuple
            same_columns = defaultdict(list) 
            
            for c in range(band.shape[1]):
                column = band[:,c]
                same_columns[tuple(column)].append(c)

            filtered_same_columns = dict()
            for k, values in same_columns.items():
                if len(values) >= 2:
                    filtered_same_columns[k] = values

            for values in filtered_same_columns.values():
                for pair in combinations(values, 2):
                    candidate_pairs.add(pair)

        return candidate_pairs

    def _check_threshold_on_signature(
        self, 
        candidate_pairs: list[tuple[int, int]], 
        signature: np.ndarray, 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            x_col = signature[:,x]
            y_col = signature[:,y]
            similarity = sum(x_col == y_col) / signature.shape[0]
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

    def check_threshold_on_cm(
        self,
        candidate_pairs: list[tuple[int, int]], 
        docs_dict: dict[int, np.ndarray], 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            similarity = jaccard_similarity(docs_dict[x], docs_dict[y])
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

In [93]:
from scipy.optimize import fsolve
from math import exp

def equations(vars):
    x, y = vars
    eq1 = x+y**2-4
    eq2 = exp(x) + x*y - 3
    return [eq1, eq2]

x, y =  fsolve(equations, (1, 1))

print(x, y)

0.6203445234801195 1.8383839306750887


In [102]:
n = 100
t = 0.5

def equations(vars):
    b, r = vars
    eq1 = t - (1 / b) ** (1 / r)
    eq2 = n - b * r
    return [eq1, eq2]

b, r =  fsolve(equations, (1, 1))

print(b, r)

22.32008145851344 4.48027038726856


In [99]:
b * r

99.99999999999982

In [103]:
from sympy import *

n = 100
b, r = symbols('b, r')
eq1 = Eq(0.5, (1 / b) ** (1 / r))
eq2 = Eq(b * r, n)

solution = solve([eq1, eq2], [b, r])
print(solution)
b, r = solution[0]
b = np.floor(b)
r = n // b
int(b), int(r)

[]


IndexError: list index out of range

In [104]:
b, r = symbols('b, r')
nsolve([eq1, eq2], [b, r], [22, 4])

Matrix([
[22.3200814585134],
[4.48027038726857]])

In [105]:
50 ** 5

312500000

In [106]:
50 ** 6

15625000000

In [21]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm', disable=['parser'], exclude=['ner'])

In [24]:
nlp = spacy.load('en_core_web_sm')

In [22]:
doc = "#standwithukraine️ #ukraineunderattack what's the best policy 62,000 for the future of 🇺🇸🇨🇳 relations over taiwan?"

In [23]:
for tok in nlp(doc):
    print(tok, tok.pos_, tok.lemma_, tok.is_punct, tok.is_stop)

#  # True False
standwithukraine️  standwithukraine️ False False
#  # True False
ukraineunderattack  ukraineunderattack False False
what  what False True
's  's False True
the  the False True
best  best False False
policy  policy False False
62,000  62,000 False False
for  for False True
the  the False True
future  future False False
of  of False True
🇺  🇺 False False
🇸  🇸 False False
🇨  🇨 False False
🇳  🇳 False False
relations  relations False False
over  over False True
taiwan  taiwan False False
?  ? True False


d:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [25]:
nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
  'parser': [],
  'attribute_ruler': [],
  'lemmatizer': [],
  'ner': []},
 'att

In [136]:
ss = "'𝙂','𝙖''𝙘''𝙙''𝙚 '𝙜''𝙝 '𝙞'𝙣' '𝙤' '𝙧' '𝙨''𝙩'𝙬'"
ss2 = "нетвойне"
ss3 = "'𝕋','𝕒''𝕕''𝕖''𝕗''𝕘''𝕙' '𝕟''𝕠''𝕣' '𝕤' '𝕥''𝕩' '𝕮''𝕹'"

In [137]:
unicodedata.normalize('NFKD', ss3)

"'T','a''d''e''f''g''h' 'n''o''r' 's' 't''x' 'C''N'"

In [142]:
"#".isprintable()

True

In [2]:
'hello'.__hash__()

1236360530447794878

In [3]:
import hashlib

In [20]:
int.from_bytes(hashlib.sha256("my_string".encode()).hexdigest()[:10], 'little')

TypeError: cannot convert 'str' object to bytes

In [27]:
b = bin(int.from_bytes(hashlib.sha256("my_string".encode()).digest(), 'little'))[-30:]
b

'101001001001110101110111110100'

In [28]:
int(b, 2)

690445812

In [30]:
def get_variable_length_hash(n: int):
    def inner_f(s: str):
        binary_str = bin(
            int.from_bytes(
                hashlib.sha256(s.encode()).digest(), 
                'little'
            )
        )[-n:]
        return int(binary_str, 2)
    return inner_f

In [31]:
f = get_variable_length_hash(30)

In [32]:
f("my_string")

690445812